In [1]:
import time, os, smtplib
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, timedelta
from dotenv import load_dotenv
load_dotenv()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def data_clean(s):
    start = time.time()
    email = os.getenv('notification_email')
    password = os.getenv('notification_password')
    players = pd.read_csv('../data/database-players.csv')
    players = players[players['Last_Year'] >= s]
    nfl_weeks = pd.read_csv('../data/NFL-Week-Dates.csv')
    nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
    
    def pre_thu(d):
        days_behind = 3 - d.weekday()
        if days_behind > 0:
            days_behind -= 7
        return d + dt.timedelta(days_behind)
    
    def findnth(haystack, needle, n):
        parts = haystack.split(needle, n+1)
        if len(parts) <= n+1:
            return -1
        return len(haystack)-len(parts[-1])-len(needle)
    
    def renaming_cols_stats(col):
        if "Unnamed:" in col:
            new = col[findnth(col, "'", 2)+1:-2]
        elif "Passing" in col or "Rushing" in col or "Receiving" in col or "Fumbles" in col or "Scoring" in col or "Tackles" in col or "Punting" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Punt Returns" in col or "ST Snaps" in col or "Kick Returns" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Def. Snaps" in col or "Off. Snaps" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)-1] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Def Interceptions" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, " ", 0)+4] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        else:
            new = col
        return new
    
    def renaming_cols_fantasy(col):
        if "Passing" in col or "Rushing" in col or "Receiving" in col or "Fumbles" in col or "Scoring" in col or "Tackles" in col or "Punting" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Punt Returns" in col or "ST Snaps" in col or "Kick Returns" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Def. Snaps" in col or "Off. Snaps" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)-1] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Def Interceptions" in col:
            new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, " ", 0)+4] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
        elif "Unnamed:" in col:
            new = col[findnth(col, "'", 2)+1:-2]
        else:
            new = col
        return new
    
    try:
        # player stats clean
        stats = pd.read_csv(f'../data/raw-data/player-weekly-stats-{s}-raw.csv', low_memory=False)
        cols = []
        for c in stats.columns:
            cols.append(c)
        cols.remove('Player_Address')
        cols.remove("('Unnamed: 3_level_0', 'Week')")
        bridge = stats.groupby(['Player_Address', "('Unnamed: 3_level_0', 'Week')"])[cols].apply(lambda x: x.ffill().bfill())
        stats.loc[:,cols] = bridge.loc[:,cols]
        stats.drop_duplicates(inplace=True)
        stats = stats.reset_index(drop=True)
        stats = stats.rename(columns=renaming_cols_stats)
        stats = stats.rename(columns = {"Unnamed: 6_level_1": "Home_Away", 
                                        "Tm": "Team", 
                                        "Passing_Yds.1": "Passing_Sk_Yds", 
                                        "GS": "Gm_Start", 
                                        "G#": "Gm_Num"})
        stats = pd.merge(left=stats, 
                         right=players, 
                         how='outer', 
                         left_on='Player_Address', 
                         right_on='Address')
        stats = stats.groupby(stats.columns, axis=1).sum()
        stats.drop(['First_Year', 'Last_Year', 'Rk', 'Address'], axis=1, inplace=True)
        stats.replace({'Home_Away': {'@': 'Away', 0: 'Home'}, 
                       'Gm_Start': {'*': True, 0: False}}, inplace=True)
        stats.dropna(subset=['Player_Address'], inplace=True)
        stats[['Off_Snaps_Pct', 
               'Def_Snaps_Pct', 
               'ST_Snaps_Pct',
               'Player_Address', 
               'Week']] = stats[['Off_Snaps_Pct', 
                                 'Def_Snaps_Pct', 
                                 'ST_Snaps_Pct', 
                                 'Player_Address', 
                                 'Week']].astype(str)
        stats['Off_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Off_Snaps_Pct'].values))
        stats['Def_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Def_Snaps_Pct'].values))
        stats['ST_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['ST_Snaps_Pct'].values))
        stats['Date'] = pd.to_datetime(stats['Date'], errors='coerce', format='%Y-%m-%d')
        stats = stats[['Player_Address', 'Player', 'Position', 'Age', 'Team', 'Home_Away', 
                       'Opp', 'Result', 'Week', 'Gm_Num', 'Season', 'Gm_Start', 'Date', 
                       'Off_Snaps_Num', 'Off_Snaps_Pct', 'Def_Snaps_Num', 'Def_Snaps_Pct',
                       'ST_Snaps_Num', 'ST_Snaps_Pct', 'Passing_Att',  'Passing_Cmp',
                       'Passing_Cmp%', 'Passing_Yds', 'Passing_TD', 'Passing_Int',
                       'Passing_Rate', 'Passing_Sk', 'Passing_Sk_Yds', 'Passing_Y/A',
                       'Passing_AY/A', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Y/A',
                       'Rushing_TD', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 
                       'Receiving_TD', 'Receiving_Y/R', 'Receiving_Ctch%', 'Receiving_Y/Tgt', 
                       'Kick_Returns_Rt', 'Kick_Returns_Yds', 'Kick_Returns_Y/Rt', 
                       'Kick_Returns_TD', 'Punt_Returns_Ret', 'Punt_Returns_Yds',
                       'Punt_Returns_TD', 'Punt_Returns_Y/R', 'Punting_Pnt', 'Punting_Yds', 
                       'Punting_Y/P', 'Punting_Blck', 'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM',
                       'Scoring_XPA', 'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA', 'Scoring_FG%',
                       'Scoring_2PM', 'Scoring_Sfty', 'Sk', 'Tackles_Solo', 'Tackles_Ast', 
                       'Tackles_Comb', 'Tackles_TFL', 'Tackles_QBHits', 'Def_Int_Int', 
                       'Def_Int_Yds', 'Def_Int_TD', 'Def_Int_PD', 'Fumbles_Fmb', 'Fumbles_FL',
                       'Fumbles_FF', 'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD']]
        for c in stats.columns[13:]:
            stats[c] = pd.to_numeric(stats[c], errors='coerce')
        stats.drop_duplicates(['Player_Address', 'Week'], inplace=True, keep='first')
        stats.to_csv(f'../data/clean-data/stats-{s}.csv', index=False)

        # player fantasy clean
#         stats = pd.read_csv(f'../data/raw-data/player-weekly-stats-{s}-raw.csv', low_memory=False)
#         cols = []
#         for c in stats.columns:
#             cols.append(c)
#         cols.remove('Player_Address')
#         cols.remove("('Unnamed: 3_level_0', 'Week')")
#         bridge = stats.groupby(['Player_Address', "('Unnamed: 3_level_0', 'Week')"])[cols].apply(lambda x: x.ffill().bfill())
#         stats.loc[:,cols] = bridge.loc[:,cols]
#         stats.drop_duplicates(inplace=True)
#         stats = stats.reset_index(drop=True)
#         stats = stats.rename(columns=renaming_cols_stats)
#         stats = stats.rename(columns = {"Unnamed: 6_level_1": "Home_Away", 
#                                         "Tm": "Team", 
#                                         "Passing_Yds.1": "Passing_Sk_Yds", 
#                                         "GS": "Gm_Start", 
#                                         "G#": "Gm_Num"})
#         stats = pd.merge(left=stats, 
#                          right=players, 
#                          how='outer', 
#                          left_on='Player_Address', 
#                          right_on='Address')
#         stats = stats.groupby(stats.columns, axis=1).sum()
#         stats.drop(['First_Year', 'Last_Year', 'Rk', 'Address'], axis=1, inplace=True)
#         stats.replace({'Home_Away': {'@': 'Away', 0: 'Home'}, 
#                        'Gm_Start': {'*': True, 0: False}}, inplace=True)
#         stats.dropna(subset=['Player_Address'], inplace=True)
#         stats[['Off_Snaps_Pct', 
#                'Def_Snaps_Pct', 
#                'ST_Snaps_Pct',
#                'Player_Address', 
#                'Week']] = stats[['Off_Snaps_Pct', 
#                                  'Def_Snaps_Pct', 
#                                  'ST_Snaps_Pct', 
#                                  'Player_Address', 
#                                  'Week']].astype(str)
#         stats['Off_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Off_Snaps_Pct'].values))
#         stats['Def_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Def_Snaps_Pct'].values))
#         stats['ST_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['ST_Snaps_Pct'].values))
#         stats['Date'] = pd.to_datetime(stats['Date'], errors='coerce', format='%Y-%m-%d')
#         stats = stats[['Player_Address', 'Player', 'Position', 'Age', 'Team', 'Home_Away', 
#                        'Opp', 'Result', 'Week', 'Gm_Num', 'Season', 'Gm_Start', 'Date', 
#                        'Off_Snaps_Num', 'Off_Snaps_Pct', 'Def_Snaps_Num', 'Def_Snaps_Pct',
#                        'ST_Snaps_Num', 'ST_Snaps_Pct', 'Passing_Att',  'Passing_Cmp',
#                        'Passing_Cmp%', 'Passing_Yds', 'Passing_TD', 'Passing_Int',
#                        'Passing_Rate', 'Passing_Sk', 'Passing_Sk_Yds', 'Passing_Y/A',
#                        'Passing_AY/A', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Y/A',
#                        'Rushing_TD', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 
#                        'Receiving_TD', 'Receiving_Y/R', 'Receiving_Ctch%', 'Receiving_Y/Tgt', 
#                        'Kick_Returns_Rt', 'Kick_Returns_Yds', 'Kick_Returns_Y/Rt', 
#                        'Kick_Returns_TD', 'Punt_Returns_Ret', 'Punt_Returns_Yds',
#                        'Punt_Returns_TD', 'Punt_Returns_Y/R', 'Punting_Pnt', 'Punting_Yds', 
#                        'Punting_Y/P', 'Punting_Blck', 'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM',
#                        'Scoring_XPA', 'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA', 'Scoring_FG%',
#                        'Scoring_2PM', 'Scoring_Sfty', 'Sk', 'Tackles_Solo', 'Tackles_Ast', 
#                        'Tackles_Comb', 'Tackles_TFL', 'Tackles_QBHits', 'Def_Int_Int', 
#                        'Def_Int_Yds', 'Def_Int_TD', 'Def_Int_PD', 'Fumbles_Fmb', 'Fumbles_FL',
#                        'Fumbles_FF', 'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD']]
#         for c in stats.columns[13:]:
#             stats[c] = pd.to_numeric(stats[c], errors='coerce')
#         stats.drop_duplicates(['Player_Address', 'Week'], inplace=True, keep='first')
#         stats.to_csv(f'../data/clean-data/stats-{s}.csv', index=False)        
        
        # player injurys clean
        injury = pd.read_csv(f'../data/raw-data/nfl-injury-report-{s}-raw.csv', low_memory=False)
        cols = [col for col in injury.columns if "Player_Address" in col]
        injury["Player_Address"] = injury[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
        injury = injury.drop(cols[1:], axis=1)
        injury = pd.melt(injury,
                         id_vars=['Player', 'Team', 'Season', 'Player_Address'], 
                         var_name='Date', 
                         value_name='Status')
        injury[['Date','Opp']] = injury.Date.str.split('vs. ', expand=True)
        injury[['Month','Day']] = injury.Date.str.split('/', expand=True)
        injury[['Status','Injury']] = injury.Status.str.split(":", expand=True)
        injury.dropna(axis=0, subset=['Status','Injury'], how='all', inplace=True)
        injury[['Season', 'Month', 'Day']] = injury[['Season', 'Month', 'Day']].astype(int)
        injury['Date'] = injury['Date'] + '/' + (np.where(injury['Month'] <= 2, injury['Season'] + 1, injury['Season'])).astype(str)
        injury['Date'] = pd.to_datetime(injury['Date'])
        injury['week_start_nfl'] = injury['Date'].apply(pre_thu)
        nfl_weeks['Start Date'] = pd.to_datetime(nfl_weeks['Start Date'])
        injury = pd.merge(left=injury, 
                          right=nfl_weeks, 
                          how='left', 
                          left_on='week_start_nfl', 
                          right_on='Start Date')
        injury.replace({'Team':
                           {'crd': 'ARI', 'atl': 'ATL', 'rav': 'BAL', 'buf': 'BUF', 
                            'car': 'CAR', 'chi': 'CHI', 'cin': 'CIN', 'cle': 'CLE', 
                            'dal': 'DAL', 'den': 'DEN', 'det': 'DET', 'gnb': 'GNB',
                            'htx': 'HOU', 'clt': 'IND', 'jax': 'JAX', 'kan': 'KAN', 
                            'sdg': 'LAC', 'ram': 'LAR', 'mia': 'MIA', 'min': 'MIN', 
                            'nor': 'NOR', 'nwe': 'NWE', 'nyg': 'NYG', 'nyj': 'NYJ', 
                            'rai': 'OAK', 'phi': 'PHI', 'pit': 'PIT', 'sea': 'SEA',
                            'sfo': 'SFO', 'tam': 'TAM', 'oti': 'TEN', 'was': 'WAS'}},
                       inplace=True)
        injury['Injury'] = injury['Injury'].str.strip(' ')
        injury.replace({"Player_Address": {"_": "", "nan": ""}}, regex=True, inplace=True)
        injury = pd.merge(left=injury,  
                          right=players, 
                          how='outer', 
                          left_on='Player_Address',
                          right_on='Address')
        injury.rename(columns={"Player_x": "Player"}, inplace=True)
        injury.drop(['Month', 'Day', 'week_start_nfl', 'Start Date', 'First_Year', 'Last_Year', 'Player_y'], axis=1, inplace=True)
        injury['Date'] = pd.to_datetime(injury['Date'], errors='coerce', format='%Y-%m-%d')
        injury.dropna(subset=['Player_Address'], inplace=True)
        injury.dropna(subset=['Week'], inplace=True)
        injury = injury[["Player", "Team", "Season", "Player_Address", "Date", "Status", "Opp", "Injury", "Week", "Position"]]
        injury.to_csv(f'../data/clean-data/injury-{s}.csv', index=False)

        # combine stats and injuries
        stats = pd.read_csv(f'../data/clean-data/stats-{s}.csv')
        injury = pd.read_csv(f'../data/clean-data/injury-{s}.csv')
        season_data = stats.merge(injury, 
                                  how="outer",
                                  on=["Player_Address", "Date"])
        season_data["Player_x"] = season_data["Player_x"].fillna(season_data["Player_y"])
        season_data["Position_x"] = season_data["Position_x"].fillna(season_data["Position_y"])
        season_data["Team_x"] = season_data["Team_x"].fillna(season_data["Team_y"])
        season_data["Opp_x"] = season_data["Opp_x"].fillna(season_data["Opp_y"])
        season_data["Week_x"] = season_data["Week_x"].fillna(season_data["Week_y"])
        season_data["Season_x"] = season_data["Season_x"].fillna(season_data["Season_y"])
        season_data = season_data.rename(columns={"Player_x": "Player", 
                                                  "Position_x": "Position", 
                                                  "Team_x": "Team", 
                                                  "Opp_x": "Opp", 
                                                  "Date_x": "Date", 
                                                  "Season_x": "Season", 
                                                  "Week_x": "Week"})
        season_data = season_data[['Player_Address', 'Player', 'Position', 'Age', 'Team', 'Home_Away', 
                                   'Opp', 'Result', 'Date', 'Week', 'Gm_Num', 'Season', 'Status', 
                                   'Injury', 'Gm_Start', 'Off_Snaps_Num', 'Off_Snaps_Pct', 
                                   'Def_Snaps_Num', 'Def_Snaps_Pct', 'ST_Snaps_Num', 'ST_Snaps_Pct', 
                                   'Passing_Att',  'Passing_Cmp', 'Passing_Cmp%', 'Passing_Yds', 
                                   'Passing_TD', 'Passing_Int', 'Passing_Rate', 'Passing_Sk', 
                                   'Passing_Sk_Yds', 'Passing_Y/A', 'Passing_AY/A', 'Rushing_Att', 
                                   'Rushing_Yds', 'Rushing_Y/A', 'Rushing_TD', 'Receiving_Tgt', 
                                   'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD', 'Receiving_Y/R', 
                                   'Receiving_Ctch%', 'Receiving_Y/Tgt', 'Kick_Returns_Rt', 
                                   'Kick_Returns_Yds', 'Kick_Returns_Y/Rt', 'Kick_Returns_TD', 
                                   'Punt_Returns_Ret', 'Punt_Returns_Yds', 'Punt_Returns_TD', 
                                   'Punt_Returns_Y/R', 'Punting_Pnt', 'Punting_Yds', 'Punting_Y/P',
                                   'Punting_Blck', 'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM',
                                   'Scoring_XPA', 'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA',
                                   'Scoring_FG%', 'Scoring_2PM', 'Scoring_Sfty', 'Sk', 'Tackles_Solo',                           
                                   'Tackles_Ast', 'Tackles_Comb', 'Tackles_TFL', 'Tackles_QBHits', 
                                   'Def_Int_Int', 'Def_Int_Yds', 'Def_Int_TD', 'Def_Int_PD', 'Fumbles_Fmb',
                                   'Fumbles_FL', 'Fumbles_FF', 'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD']]
        season_data.to_csv(f'../data/clean-data/season-data-{s}.csv', index=False)

        # create databases for healthy weeks and injured weeks
        season_data = pd.read_csv(f'../data/clean-data/season-data-{s}.csv', low_memory=False)
        season_data.sort_values(['Player_Address','Week'], inplace=True)
        season_data.reset_index(inplace=True)
        season_data.drop(['index'], axis=1, inplace=True)
        status = season_data['Status'].dropna().unique().tolist()
        player_list = season_data.loc[season_data['Status'].isin(status)].drop_duplicates(subset=['Player_Address']).index.unique().tolist()
        injury_weeks = pd.DataFrame()
        healthy_weeks = pd.DataFrame()
        for i in range(0,len(player_list)):
            idx = season_data[season_data['Player_Address'] == season_data['Player_Address'][player_list[i]]].tail(1)
            inj = season_data.loc[player_list[i]-1: idx.index.values.astype(int)[0]]
            injury_weeks = pd.concat([injury_weeks, inj])
        injury_weeks = injury_weeks.drop_duplicates(['Player_Address', 'Week'],keep='first')
        injury_weeks['Inj_Week'] = 0
        injury_weeks['Inj_Week'] = injury_weeks.groupby('Player_Address').cumcount()
        for i in range(0,len(player_list)):
            idx = season_data[season_data['Player_Address'] == season_data['Player_Address'][player_list[i]]].head(1)
            healthy = season_data.loc[idx.index.values.astype(int)[0]:player_list[i]-1]
            healthy_weeks = pd.concat([healthy_weeks, healthy])
        healthy_weeks['Healthy_Week'] = 0
        healthy_weeks['Healthy_Week'] = healthy_weeks.groupby('Player_Address').cumcount()+1
        injury_weeks.to_csv(f'../data/clean-data/weeks-injured-{s}.csv', index=False)
        healthy_weeks.to_csv(f'../data/clean-data/weeks-healthy-{s}.csv', index=False)

        # combine injured and healthy databases
        injury_weeks = pd.read_csv(f'../data/clean-data/weeks-injured-{s}.csv')
        healthy_weeks = pd.read_csv(f'../data/clean-data/weeks-healthy-{s}.csv')
        inj_cols = injury_weeks.columns.tolist()
        heal_cols = healthy_weeks.columns.tolist()
        inj_cols.remove('Inj_Week')
        inj_cols.remove('Status')
        inj_cols.remove('Injury')
        heal_cols.remove('Healthy_Week')
        heal_cols.remove('Status')
        heal_cols.remove('Injury')
        nfl_data = injury_weeks.merge(healthy_weeks, how="outer", left_on=inj_cols, right_on=heal_cols)
        nfl_data['Status_x'] = nfl_data['Status_x'].fillna(nfl_data['Status_y'])
        nfl_data['Injury_x'] = nfl_data['Injury_x'].fillna(nfl_data['Injury_y'])
        nfl_data.rename(columns={'Status_x': 'Status',
                                 'Injury_x': 'Injury'},
                        inplace=True)
        nfl_data = nfl_data.merge(players, how="outer", left_on="Player_Address", right_on="Address")
        nfl_data["Position_x"] = nfl_data["Position_x"].fillna(nfl_data["Position_y"])
        nfl_data["Player_Address"] = nfl_data["Player_Address"].fillna(nfl_data["Address"])
        nfl_data["Player_x"] = nfl_data["Player_x"].fillna(nfl_data["Player_y"])
        nfl_data.rename(columns={"Player_x": "Player",
                                 "Position_x": "Position"},
                        inplace=True)
        nfl_data.drop(columns=['Injury_y','Status_y', 'Player_y', 'First_Year', 'Last_Year', 'Address', 'Position_y'], inplace=True)
        nfl_data.to_csv(f'../data/nfl-{s}.csv', index=False)
    except Exception as _ex:
        error = repr(_ex)
    end = time.time()
    to = "cuddebtj@gmail.com"
    subject = 'Python Script Complete: Data Clean'
    if error != "":
        body = "Error raised.\n%s" % (error)
    else:
        body = "Data-Clean done for season %s.\nTime to complete: %s" % (s, (end-start)/60)
    email_text = """\
    From: %s
    To: %s
    Subject: %s

    %s
    """ % (email, to, subject, body)
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(email, password)
    server.sendmail(sent_from, to, email_text)
    server.close()

In [ ]:
for s in range(2015,2021):
    data_clean(s)

In [87]:
def findnth(haystack, needle, n):
    parts = haystack.split(needle, n+1)
    if len(parts) <= n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

def renaming_cols_fantasy(col):
    if "Fantasy" in col:
        new = col[findnth(col, "'", 2)+1:findnth(col, "'", 3)] + "_" + col[findnth(col, "'", 4)+1:-2]
    elif "Inside 10" in col or "Inside 20" in col or "ST" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)] + "_" + col[findnth(col, "'", 4)+1:-2]
    elif "Def." in col or "Off." in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)-1] + "_" + col[findnth(col, "'", 4)+1:-2]
    elif "Unnamed:" in col:
        new = col[findnth(col, "'", 4)+1:-2]
    else:
        new = col
    return new
s = 2015
fan = pd.read_csv(f'../data/raw-data/player-weekly-fantasy-{s}-raw.csv', low_memory=False)
cols = []
players = pd.read_csv('../data/database-players.csv')
players = players[players['Last_Year'] >= s]
nfl_weeks = pd.read_csv('../data/NFL-Week-Dates.csv')
nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
for c in fan.columns:
    cols.append(c)
fan = fan.rename(columns=renaming_cols_fantasy)
fan = fan.rename(columns = {"Unnamed: 4_level_2": "Home_Away", 
                            "Tm": "Team", 
                            "GS": "Gm_Start", 
                            "G#": "Gm_Num", 
                            "Pos": "Position", 
                            "Snap Counts_Off_Num": "Off_Snaps_Num", 
                            "Snap Counts_Off_Pct": "Off_Snaps_Pct",
                            "Snap Counts_Def_Num": "Def_Snaps_Num", 
                            "Snap Counts_Def_Pct": "Def_Snaps_Pct",
                            "Snap Counts_ST_Num": "ST_Snaps_Num", 
                            "Snap Counts_ST_Pct": "ST_Snaps_Pct", 
                            "Inside 20_Rushing_Att": "In20_Rushing_Att", 
                            "Inside 20_Rushing_Yds": "In20_Rushing_Yds",
                            "Inside 20_Rushing_TD": "In20_Rushing_TD",
                            "Inside 10_Rushing_Att": "In10_Rushing_Att", 
                            "Inside 10_Rushing_Yds": "In10_Rushing_Yds",
                            "Inside 10_Rushing_TD": "In10_Rushing_TD",
                            "Inside 20_Receiving_Tgt": "In20_Receiving_Tgt",
                            "Inside 20_Receiving_Rec": "In20_Receiving_Rec",
                            "Inside 20_Receiving_Yds": "In20_Receiving_Yds",
                            "Inside 20_Receiving_TD": "In20_Receiving_TD",
                            "Inside 10_Receiving_Tgt": "In10_Receiving_Tgt",
                            "Inside 10_Receiving_Rec": "In10_Receiving_Rec",
                            "Inside 10_Receiving_Yds": "In10_Receiving_Yds",
                            "Inside 10_Receiving_TD": "In10_Receiving_TD",
                            "Inside 20_Passing_Cmp": "In20_Passing_Cmp", 
                            "Inside 20_Passing_Att": "In20_Passing_Att",
                            "Inside 20_Passing_Yds": "In20_Passing_Yds", 
                            "Inside 20_Passing_TD": "In20_Passing_TD",
                            "Inside 10_Passing_Cmp": "In10_Passing_Cmp", 
                            "Inside 10_Passing_Att": "In10_Passing_Att",
                            "Inside 10_Passing_Yds": "In10_Passing_Yds", 
                            "Inside 10_Passing_TD": "In10_Passing_TD"})
fan = pd.merge(left=fan, 
               right=players, 
               how='outer', 
               left_on='Player_Address', 
               right_on='Address')
fan = fan.groupby(fan.columns, axis=1).sum()
fan.drop(['First_Year', 'Last_Year', 'Rk', 'Address'], axis=1, inplace=True)
fan.replace({'Home_Away': {'@': 'Away', 0: 'Home'}, 
               'Gm_Start': {'*': True, 0: False}}, inplace=True)
fan.dropna(subset=['Player_Address'], inplace=True)
fan[['Off_Snaps_Pct', 
       'Def_Snaps_Pct', 
       'ST_Snaps_Pct',
       'Player_Address']] = fan[['Off_Snaps_Pct', 
                         'Def_Snaps_Pct', 
                         'ST_Snaps_Pct', 
                         'Player_Address']].astype(str)
fan['Off_Snaps_Pct'] = list(map(lambda x: x[:-1], fan['Off_Snaps_Pct'].values))
fan['Def_Snaps_Pct'] = list(map(lambda x: x[:-1], fan['Def_Snaps_Pct'].values))
fan['ST_Snaps_Pct'] = list(map(lambda x: x[:-1], fan['ST_Snaps_Pct'].values))
fan['Date'] = pd.to_datetime(fan['Date'], errors='coerce', format='%Y-%m-%d')
fan['Position_x'] = fan['Position_x'].fillna(fan['Position_y'])
fan.rename(columns={'Position_x': 'Position'},
          inplace=True)
fan = fan[['Player_Address', 'Player', 'Position', 'Team', 'Home_Away', 'Opp', 
           'Result', 'Gm_Num', 'Season', 'Date', 'Off_Snaps_Num', 'Off_Snaps_Pct', 
           'Def_Snaps_Num', 'Def_Snaps_Pct', 'ST_Snaps_Num', 'ST_Snaps_Pct', 
           'In10_Passing_Att', 'In10_Passing_Cmp', 'In10_Passing_TD', 
           'In10_Passing_Yds', 'In20_Rushing_Att', 'In20_Rushing_TD', 
           'In20_Rushing_Yds', 'In10_Receiving_Rec', 'In10_Receiving_TD', 
           'In10_Receiving_Tgt', 'In10_Receiving_Yds', 'In10_Rushing_Att', 
           'In10_Rushing_TD', 'In10_Rushing_Yds', 'In20_Passing_Att', 
           'In20_Passing_Cmp', 'In20_Passing_TD', 'In20_Passing_Yds', 
           'In20_Receiving_Rec', 'In20_Receiving_TD', 'In20_Receiving_Tgt', 
           'In20_Receiving_Yds']]

In [88]:
fan.head(10)

,Player_Address,Player,Position,Team,Home_Away,Opp,Result,Gm_Num,Season,Date,Off_Snaps_Num,Off_Snaps_Pct,Def_Snaps_Num,Def_Snaps_Pct,ST_Snaps_Num,ST_Snaps_Pct,In10_Passing_Att,In10_Passing_Cmp,In10_Passing_TD,In10_Passing_Yds,In10_Receiving_Rec,In10_Receiving_TD,In10_Receiving_Tgt,In10_Receiving_Yds,In10_Rushing_Att,In10_Rushing_TD,In10_Rushing_Yds,In20_Passing_Att,In20_Passing_Cmp,In20_Passing_TD,In20_Passing_Yds,In20_Receiving_Rec,In20_Receiving_TD,In20_Receiving_Tgt,In20_Receiving_Yds,In20_Rushing_Att,In20_Rushing_TD,In20_Rushing_Yds
0,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Away,DEN,L 13-19,1.0,2015.0,2015-09-13,58.0,100.0,0.0,0.0,3.0,10.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Away,OAK,L 33-37,2.0,2015.0,2015-09-20,82.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Home,CIN,L 24-28,3.0,2015.0,2015-09-27,74.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Away,PIT,W 23-20,4.0,2015.0,2015-10-01,79.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Home,CLE,L 30-33,5.0,2015.0,2015-10-11,70.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Away,SFO,L 20-25,6.0,2015.0,2015-10-18,76.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Away,ARI,L 18-26,7.0,2015.0,2015-10-26,64.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Home,SDG,W 29-26,8.0,2015.0,2015-11-01,43.0,63.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,/players/Z/ZuttJe20,Jeremy Zuttah,C,BAL,Home,JAX,L 20-22,9.0,2015.0,2015-11-15,66.0,97.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,/players/Z/ZuerGr00,Greg Zuerlein,K,STL,Home,SEA,W 34-31,1.0,2015.0,2015-09-13,0.0,0.0,0.0,0.0,13.0,40.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

# for c in fan.columns[13:]:
#     fan[c] = pd.to_numeric(fan[c], errors='coerce')
# fan.drop_duplicates(['Player_Address', 'Week'], inplace=True, keep='first')
# fan.to_csv(f'../data/clean-data/fantasy-{s}.csv', index=False)